In [6]:
# Surgery type attributes (previously defined)
surgery_type_attributes = {
    "Cardiovascular": {"duration": 4, "variance": 1.0, "priority": 5},
    "Neurological": {"duration": 5, "variance": 1.2, "priority": 4},
    "Orthopedic": {"duration": 3, "variance": 0.5, "priority": 3},
    "Cosmetic": {"duration": 2, "variance": 0.3, "priority": 1},
    "Gastrointestinal": {"duration": 3.5, "variance": 0.7, "priority": 2},
}

# Surgeries list
surgeries = [f"{type} Surgery {i}" for type in surgery_type_attributes for i in range(1, 11)]



In [19]:
from gurobipy import Model, GRB

# Constants
days = 10
rooms = 3
hours_per_day = 8  # Assuming each operating room is available 8 hours per day

# Model
m = Model("SurgeryScheduling")

# Decision variables: surgery_scheduled[surgery, day, room]
surgery_scheduled = m.addVars(surgeries, range(days), range(rooms), vtype=GRB.BINARY, name="schedule")

# Define a weight for each day, decreasing over time to prioritize earlier scheduling
day_weights = {d: (days - d) for d in range(days)}

# Modify the objective function to minimize late cost, incorporating surgery priority and day weight
m.setObjective(sum(surgery_scheduled[s, d, r] * surgery_type_attributes[s.split()[0]]['priority'] * day_weights[d]
                   for s in surgeries for d in range(days) for r in range(rooms)), GRB.MAXIMIZE)


# Constraint: Each surgery can only be scheduled once
for s in surgeries:
    m.addConstr(sum(surgery_scheduled[s, d, r] for d in range(days) for r in range(rooms)) == 1, f"OneTime_{s}")

# Constraint: Respect operating room availability (duration and operating hours)
for d in range(days):
    for r in range(rooms):
        m.addConstr(sum(surgery_scheduled[s, d, r] * surgery_type_attributes[s.split()[0]]['duration']
                        for s in surgeries) <= hours_per_day, f"RoomAvailability_{d}_{r}")

# Optimize model
m.optimize()

# The structure will be: surgery_name: {"day": day, "room": room}
scheduled_surgeries = {}

# Populate the dictionary based on the optimization results
for s in surgeries:
    for d in range(days):
        for r in range(rooms):
            # Check if the surgery is scheduled based on the decision variable
            if surgery_scheduled[s, d, r].X > 0.5:  # Using 0.5 as a threshold to indicate selection
                # Store the scheduled day and room for each surgery
                scheduled_surgeries[s] = {"day": d+1, "room": r+1}

# Print or use the scheduled_surgeries dictionary as needed
for surgery, info in scheduled_surgeries.items():
    print(f"{surgery} is scheduled on day {info['day']} in room {info['room']}")




Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i5-10210U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 80 rows, 1500 columns and 3000 nonzeros
Model fingerprint: 0xa74473c6
Variable types: 0 continuous, 1500 integer (1500 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+00]
  Objective range  [1e+00, 5e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+00]
Found heuristic solution: objective 832.0000000
Presolve time: 0.02s
Presolved: 80 rows, 1500 columns, 3000 nonzeros
Variable types: 0 continuous, 1500 integer (1500 binary)
Found heuristic solution: objective 1012.0000000

Root relaxation: objective 1.116400e+03, 370 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   G

In [23]:
import pandas as pd

# Assuming scheduled_surgeries is already populated with optimization results
# Convert the scheduled_surgeries dictionary to a DataFrame
df_scheduled = pd.DataFrame([(surgery, info['day'], info['room'], surgery.split()[0])
                             for surgery, info in scheduled_surgeries.items()],
                            columns=['Surgery', 'Day', 'Room', 'Type'])

# Since 'Type' is now directly taken from the surgery name, there's no need to map it again
# However, we'll update 'Duration' based on the actual surgery type names
df_scheduled['Duration'] = df_scheduled['Type'].apply(lambda x: surgery_type_attributes[x]['duration'])

# For missed surgeries, assuming a hypothetical way to identify them,
# we will continue with an empty DataFrame as no logic for identifying missed surgeries was provided
df_missed = pd.DataFrame(columns=['Type', 'Missed Count'])

# Placeholder: Assuming an example logic for missed surgeries could be applied here if available
# This example assumes no missed surgeries
for type_name in surgery_type_attributes.keys():
    missed_count = 0  # Placeholder; actual logic needed to count missed surgeries
    df_missed = df_missed.append({'Type': type_name, 'Missed Count': missed_count}, ignore_index=True)

# Example output of scheduled surgeries DataFrame
print(df_scheduled.head())

# Example output for missed surgeries DataFrame (based on placeholder logic, likely empty)
print(df_missed)


                    Surgery  Day  Room            Type  Duration
0  Cardiovascular Surgery 1    2     3  Cardiovascular       4.0
1  Cardiovascular Surgery 2    2     1  Cardiovascular       4.0
2  Cardiovascular Surgery 3    1     3  Cardiovascular       4.0
3  Cardiovascular Surgery 4    1     2  Cardiovascular       4.0
4  Cardiovascular Surgery 5    1     2  Cardiovascular       4.0
               Type Missed Count
0    Cardiovascular            0
1      Neurological            0
2        Orthopedic            0
3          Cosmetic            0
4  Gastrointestinal            0


C:\Users\Agishan\AppData\Local\Temp\ipykernel_12224\2541608762.py:21: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\Agishan\AppData\Local\Temp\ipykernel_12224\2541608762.py:21: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\Agishan\AppData\Local\Temp\ipykernel_12224\2541608762.py:21: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\Agishan\AppData\Local\Temp\ipykernel_12224\2541608762.py:21: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\Agishan\AppData\Local\Temp\ipykernel_12224\2541608762.py:21: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use

In [24]:
df_scheduled

,Surgery,Day,Room,Type,Duration
0,Cardiovascular Surgery 1,2,3,Cardiovascular,4.0
1,Cardiovascular Surgery 2,2,1,Cardiovascular,4.0
2,Cardiovascular Surgery 3,1,3,Cardiovascular,4.0
3,Cardiovascular Surgery 4,1,2,Cardiovascular,4.0
4,Cardiovascular Surgery 5,1,2,Cardiovascular,4.0
5,Cardiovascular Surgery 6,1,1,Cardiovascular,4.0
6,Cardiovascular Surgery 7,2,1,Cardiovascular,4.0
7,Cardiovascular Surgery 8,1,3,Cardiovascular,4.0
8,Cardiovascular Surgery 9,1,1,Cardiovascular,4.0
9,Cardiovascular Surgery 10,2,3,Cardiovascular,4.0


In [26]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

# Assuming df_scheduled and df_missed are already defined as per previous instructions

# Initialize Figure for Operating Room Utilization
fig = go.Figure()

# Iterate through each operating room
for room in sorted(df_scheduled['Room'].unique()):
    room_data = df_scheduled[df_scheduled['Room'] == room].groupby('Day').agg({'Duration': 'sum'}).reset_index()
    
    # Generate hover text showing the count of each surgery type per day
    hover_text = []
    for _, row in room_data.iterrows():
        day_data = df_scheduled[(df_scheduled['Room'] == room) & (df_scheduled['Day'] == row['Day'])]
        type_counts = day_data['Type'].value_counts().to_dict()
        hover_text_day = ", ".join([f"{type}: {count}" for type, count in type_counts.items()])
        hover_text.append(hover_text_day)
    
    # Calculate text for the bar labels
    text = [f"Total Surgeries: {len(day_data)}" for _, day_data in room_data.iterrows()]
    
    # Add bar chart for each operating room
    fig.add_trace(go.Bar(x=room_data['Day'], y=room_data['Duration'], name=f'Room {room}',
                         hoverinfo='text', text=text, hovertext=hover_text, textposition='auto'))

# Add an 8-hour operating limit line and other enhancements
fig.add_hline(y=8, line_dash="dot", annotation_text="8-hour limit", annotation_position="bottom right")
fig.update_layout(title_text='Operating Room Utilization by Day',
                  xaxis_title="Day",
                  yaxis_title="Hours Used",
                  legend_title_text='Operating Room',
                  legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01),
                  barmode='group',
                  bargap=0.15)

# Visualization for Missed Surgeries by Type
fig_missed = px.bar(df_missed, x='Type', y='Missed Count', text='Missed Count',
                    title='Missed Surgeries by Type',
                    labels={'Missed Count': 'Number of Missed Surgeries', 'Type': 'Surgery Type'})

# Update layout for missed surgeries chart
fig_missed.update_layout(xaxis_title="Surgery Type", yaxis_title="Number of Missed Surgeries",
                         legend_title_text='Surgery Type')

# Displaying the figures
fig.show()
fig_missed.show()
